## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\useR\Anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=100)

C:\Users\useR\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 8s 162us/step - loss: 1.3058 - acc: 0.5427
Epoch 2/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.9053 - acc: 0.6836
Epoch 3/100
50000/50000 [==============================] - 6s 125us/step - loss: 0.7590 - acc: 0.7372
Epoch 4/100
50000/50000 [==============================] - 6s 125us/step - loss: 0.6629 - acc: 0.7721
Epoch 5/100
50000/50000 [==============================] - 6s 129us/step - loss: 0.5792 - acc: 0.8005
Epoch 6/100
50000/50000 [==============================] - 6s 130us/step - loss: 0.5089 - acc: 0.8229
Epoch 7/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.4477 - acc: 0.8456
Epoch 8/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.3902 - acc: 0.8659
Epoch 9/100
50000/50000 [==============================] - 6s 121us/step - loss: 0.3443 - acc: 0.8819
Epoch 10/100
50000/50000 [==============================] - 6s 123us/step - loss: 

50000/50000 [==============================] - 6s 122us/step - loss: 0.0342 - acc: 0.9886
Epoch 80/100
50000/50000 [==============================] - 6s 123us/step - loss: 0.0298 - acc: 0.9901
Epoch 81/100
50000/50000 [==============================] - 6s 122us/step - loss: 0.0264 - acc: 0.9909
Epoch 82/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.0280 - acc: 0.9902
Epoch 83/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.0330 - acc: 0.9884
Epoch 84/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.0272 - acc: 0.9907
Epoch 85/100
50000/50000 [==============================] - 6s 121us/step - loss: 0.0254 - acc: 0.9914
Epoch 86/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.0221 - acc: 0.9925
Epoch 87/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.0222 - acc: 0.9925
Epoch 88/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.02

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.9325055e-10, 2.2543016e-20, 4.1391857e-02, 9.4442493e-01,
        1.4180411e-02, 1.6609909e-09, 2.8446468e-06, 4.4879447e-09,
        7.2578588e-09, 1.3412018e-17]], dtype=float32)